In [1]:
!pip install hazm
import pandas as pd
from __future__ import unicode_literals
from hazm import *
import re
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 233 kB 31.4 MB/s 
     |████████████████████████████████| 1.4 MB 35.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=f7af66000276d254e7ff15b19ae2fa056c37f6b3de39e8641041057b05a79935
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=155178 sha256=443b3d5a4dc29b3a5d51bb081e8ed97e1b45cf27098d5e2548843195637e837b
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
def acronyms(txt):
  normalizer = Normalizer()
  txt=normalizer.normalize(txt)
  txt = re.sub(r"[A-Za-z«»,:;</()\[\d+>]", "", txt)
  #print(txt)
  matches = re.findall("([\s]([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهیئ۱۲۳۴۵۶۷۸۹۰| ]{1,6}\.){2,4})", txt)
  for macth in matches:
  #  print(macth[0])
    y= macth[0]
    z= y.replace('. ', '\u200c')
    z= z.replace('.', '')
  #  print(z)
    txt = txt.replace(y, z)
  #  print(txt)

  return txt

In [4]:


linesnew = []
output = open('voa_fa.txt', "w", encoding = "utf-8", errors='ignore') 
with open("voa_fa_2003-2008_orig.txt", "r", encoding = "utf-8", errors='ignore') as f:
    line = f.readline()
    while line:
     
        line = f.readline()
        line=acronyms(line)
        linesnew.append(line)
        output.write(line)


output.close()

In [5]:
  print(linesnew[3])

در یکی از این گزارشها، که از سوی خبرگزاری کیودو، انتشار یافته، گفته شده است که دو کشور برای رفع اختلافات دیرین خود بر سر چهار جزیره از جزایر زنجیره‌ای کوریل، بر اساس سه پیمان گذشته خود عمل خواهند کرد. بموجب یکی از این پیمانها که در سال  امضاء شده، دو تا از این جزیره‌ها پس از امضاء یک پیمان صلح به ژاپن پس داده خواهد شد. اما بموجب پیمانی که در سال  به امضاء رسیده، مسئله حاکمیت این چهار جزیره بایستی پیش از امضاء پیمان صلح فیصله یابد. هیچ یک از دو طرف نحوه استفاده از پیمان‌های پیشین را اعلام نکرده‌اند. 



In [6]:
vocabulary = []
for sentence in linesnew:
    sentence= word_tokenize(sentence) 
    for token in sentence:
       
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)


In [15]:
vocabulary[35000]

'بلاگ\u200cهای'

In [18]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in linesnew:
    sentence= word_tokenize(sentence) 
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [19]:
idx_pairs

array([[  0,   1],
       [  1,   0],
       [  0,   2],
       ...,
       [562,  32],
       [ 32, 681],
       [ 32, 562]])

In [20]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [ ]:

embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 100
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.item()
        #total_loss += loss_val.data
        
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')